In [32]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

categories = ["FoodTrucks", "NotFoodTruck"]
training_data = []

def create_training_data():
    for category in categories:
        class_num = categories.index(category) 
        path = os.path.join("/tmp/Data/", category)

        for img in tqdm(os.listdir(path)): 
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (150, 150))
            training_data.append([new_array, class_num])

create_training_data()

random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, 150, 150, 1)

X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten()) 
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

y = np.array(y)

model.fit(X, y, batch_size=5, epochs=10)
# model.summary()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model.fit(X_train, y_train, batch_size=10, epochs=1)

# test_loss, test_acc = model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)

# model.save('model.h5')

100%|██████████| 30/30 [00:00<00:00, 1878.33it/s]


Epoch 1/10
12/12 [==============================] - 26s 2s/step - loss: 5.8004 - accuracy: 0.4746
Epoch 2/10
12/12 [==============================] - 25s 2s/step - loss: 0.6908 - accuracy: 0.5593
Epoch 3/10
12/12 [==============================] - 25s 2s/step - loss: 0.7074 - accuracy: 0.5932
Epoch 4/10
12/12 [==============================] - 25s 2s/step - loss: 0.6000 - accuracy: 0.7119
Epoch 5/10
12/12 [==============================] - 25s 2s/step - loss: 0.4686 - accuracy: 0.8136
Epoch 6/10
12/12 [==============================] - 29s 2s/step - loss: 0.2607 - accuracy: 0.9153
Epoch 7/10
12/12 [==============================] - 25s 2s/step - loss: 0.0994 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 25s 2s/step - loss: 0.0243 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 25s 2s/step - loss: 0.0125 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 25s 2s/step - loss: 0.0071 - accuracy: 1.0000


In [ ]:
# This part of the code is for testing the model
model = tf.keras.models.load_model('model.h5')

def classify_image(image_path):
    # read the image
    img = cv2.imread(image_path)
    # convert to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # resize to 150x150
    resized_img = cv2.resize(gray_img, (150, 150))
    # normalize the image
    resized_img = resized_img/255.0
    # reshape the image to match the model input shape
    resized_img = resized_img.reshape(-1, 150, 150, 1)
    # use the model to predict the class of the image
    prediction = model.predict(resized_img)
    # interpret the prediction
    if prediction[0][0] > 0.5:
        return True
    return False

def test_images(folder_path):
    correct_count = 0
    total_count = 0
    for img in os.listdir(folder_path):
        file_path = os.path.join(folder_path, img)
        expected_output = not ("not" in img)
        output = classify_image(file_path)
        if output == expected_output:
            correct_count += 1
        total_count += 1
    
    accuracy = correct_count / total_count
    print(f'Accuracy: {accuracy}')

#This is an example of how you could use these functions to test the model and make sure its predictions are correct
folder_path = "/tmp/Data/Test"
test_images(folder_path)
